In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import gc

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import (GRU, LSTM, Bidirectional, GlobalMaxPool1D, Dropout, TimeDistributed, Flatten, Activation, 
                          Concatenate, Multiply, RepeatVector, Permute, Lambda, BatchNormalization, Add, CuDNNGRU, CuDNNLSTM, Conv1D, MaxPooling1D)
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import backend as K
from keras import regularizers
from Attention import Attention
import tensorflow as tf

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import words, stopwords

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import log_loss
import seq2seq

from tqdm import tqdm
import hunspell
import editdistance #Levenshtein distance

from joblib import Parallel, delayed
import multiprocessing

In [2]:
def metric(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    columns = y_true.shape[1]
    column_losses = []
    for i in range(0, columns):
        column_losses.append(log_loss(y_true[:, i], y_pred[:, i]))
    return np.array(column_losses).mean()

In [3]:
max_words = 100000
maxlen = 150

train = pd.read_csv("./train_cleaned.csv")
test = pd.read_csv("./test_cleaned.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

tokenizer = text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list(list_sentences_train))
word_index = tokenizer.word_index
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [4]:
embeddings_index = {}
embed_size = 300

f = open('./glove.840B.300d.txt') #Use this as our list of valid words
valid_words = []
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 2195895 word vectors.


In [5]:
num_words = min(max_words, len(word_index))
embedding_matrix = np.zeros((num_words, embed_size))

for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [6]:
def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_words, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Dropout(0.25)(x)
    #x = Bidirectional(CuDNNLSTM(300, return_sequences=True))(x)
    
    
    f = CuDNNLSTM(300, return_sequences=True)(x)
    f= Attention(maxlen)(f)
    b = CuDNNLSTM(300, return_sequences=True, go_backwards=True)(x)
    b= Attention(maxlen)(b)

    x = Concatenate()([f,b])
    x = Dropout(0.25)(x)
    
    #x = GlobalMaxPool1D()(x)
    
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    return model


model = get_model()
init_weights = model.get_weights()
model.summary()
plot_model(model, to_file='model.png',show_shapes=True)

file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 150, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        (None, 150, 300)     722400      dropout_1[0][0]                  
__________________________________________________________________________________________________
cu_dnnlstm

In [7]:
#Stratify KFold by creating unique labels for ground truth values, toss things with less than num_folds into an 'others' label
num_folds = 10
truth_labels = np.unique(y,axis=0)

truth_counts = np.zeros(truth_labels.shape[0],dtype=int)
for j, tlab in enumerate(truth_labels):
    truth_counts[j] = (y == tlab).all(-1).sum()

other_label = truth_labels.shape[0]+1 #create a new label out of the index range for 'others'
truth_encoded = []
for row in y:
    for j, truth_row in enumerate(truth_labels):
        index_test = np.array_equal(row, truth_row)
        if index_test:
            if truth_counts[j]>=num_folds:
                truth_encoded.append(j)
            else:
                truth_encoded.append(other_label)

In [8]:
batch_size = 512
epochs = 50
kf = StratifiedKFold(n_splits=num_folds, random_state=42, shuffle=True)
y_test = np.zeros((len(X_te),6))
valpred = np.zeros((len(X_t),6))
for j, (train_index, val_index) in enumerate(kf.split(X_t, truth_encoded)):
    print("Training on fold {:d}".format(j))
    #model.set_weights(init_weights)
    K.clear_session()
    model = get_model()
    early = EarlyStopping(monitor="val_loss", patience=10)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',  patience=3, verbose=1, factor=0.5, min_lr=0.0001)
    checkpoint = ModelCheckpoint("fold_" + str(j) + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    history = model.fit(X_t[train_index], y[train_index], batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(X_t[val_index], y[val_index]), callbacks=[checkpoint, early, learning_rate_reduction])
    
    model.load_weights("fold_" + str(j) + ".hdf5")
    
    valpred[val_index,:]=model.predict(X_t[val_index], verbose=1, batch_size=512)
    logloss = metric(y[val_index],valpred[val_index,:])
    print("Loss on fold {:d} is {:f}".format(j,logloss))
    y_test += model.predict(X_te, verbose=1, batch_size=512)
    
y_test = y_test/num_folds
logloss = metric(y,valpred)
print("CV loss is {:f}".format(logloss))
sample_submission = pd.read_csv("./sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("submission.csv", index=False)

temp = pd.read_csv("./train.csv")
temp[list_classes] = valpred
temp.to_csv("validation_predictions.csv", index = False)

Training on fold 0
Train on 86253 samples, validate on 9598 samples
Epoch 1/50
86253/86253 [==============================] - 64s 737us/step - loss: 0.2420 - acc: 0.9414 - val_loss: 0.1398 - val_acc: 0.9633
Epoch 2/50
86253/86253 [==============================] - 63s 734us/step - loss: 0.0555 - acc: 0.9802 - val_loss: 0.0812 - val_acc: 0.9703
Epoch 3/50
86253/86253 [==============================] - 63s 736us/step - loss: 0.0500 - acc: 0.9815 - val_loss: 0.0515 - val_acc: 0.9804
Epoch 4/50
86253/86253 [==============================] - 63s 734us/step - loss: 0.0472 - acc: 0.9822 - val_loss: 0.0476 - val_acc: 0.9825
Epoch 5/50
86253/86253 [==============================] - 63s 732us/step - loss: 0.0448 - acc: 0.9830 - val_loss: 0.0427 - val_acc: 0.9837
Epoch 6/50
86253/86253 [==============================] - 63s 725us/step - loss: 0.0430 - acc: 0.9836 - val_loss: 0.0440 - val_acc: 0.9830
Epoch 7/50
86253/86253 [==============================] - 63s 731us/step - loss: 0.0412 - acc: 0.9

Epoch 10/50
86258/86258 [==============================] - 62s 722us/step - loss: 0.0376 - acc: 0.9852 - val_loss: 0.0454 - val_acc: 0.9834
Epoch 11/50
86258/86258 [==============================] - 62s 724us/step - loss: 0.0360 - acc: 0.9858 - val_loss: 0.0463 - val_acc: 0.9826
Epoch 12/50
86258/86258 [==============================] - 63s 730us/step - loss: 0.0353 - acc: 0.9861 - val_loss: 0.0434 - val_acc: 0.9841
Epoch 13/50
86258/86258 [==============================] - 63s 727us/step - loss: 0.0338 - acc: 0.9865 - val_loss: 0.0444 - val_acc: 0.9837
Epoch 14/50
86258/86258 [==============================] - 63s 730us/step - loss: 0.0325 - acc: 0.9870 - val_loss: 0.0453 - val_acc: 0.9837
Epoch 15/50
86258/86258 [==============================] - 63s 733us/step - loss: 0.0312 - acc: 0.9875 - val_loss: 0.0485 - val_acc: 0.9819
Epoch 16/50
86016/86258 [============================>.] - ETA: 0s - loss: 0.0298 - acc: 0.9881Epoch 00016: val_loss did not improve

Epoch 00016: reducing lear

9588/9588 [==============================] - 2s 228us/step
Loss on fold 3 is 0.041691
226998/226998 [==============================] - 50s 222us/step
Training on fold 4
Train on 86264 samples, validate on 9587 samples
Epoch 1/50
86264/86264 [==============================] - 63s 734us/step - loss: 0.2808 - acc: 0.9247 - val_loss: 0.0974 - val_acc: 0.9680
Epoch 2/50
86264/86264 [==============================] - 63s 727us/step - loss: 0.0620 - acc: 0.9789 - val_loss: 0.0774 - val_acc: 0.9751
Epoch 3/50
86264/86264 [==============================] - 63s 726us/step - loss: 0.0534 - acc: 0.9805 - val_loss: 0.0655 - val_acc: 0.9797
Epoch 4/50
86264/86264 [==============================] - 63s 735us/step - loss: 0.0497 - acc: 0.9814 - val_loss: 0.0488 - val_acc: 0.9815
Epoch 5/50
86264/86264 [==============================] - 63s 728us/step - loss: 0.0466 - acc: 0.9823 - val_loss: 0.0491 - val_acc: 0.9827
Epoch 6/50
86264/86264 [==============================] - 63s 731us/step - loss: 0.0444

Epoch 6/50
86268/86268 [==============================] - 63s 727us/step - loss: 0.0432 - acc: 0.9835 - val_loss: 0.0439 - val_acc: 0.9831
Epoch 7/50
86268/86268 [==============================] - 62s 724us/step - loss: 0.0416 - acc: 0.9839 - val_loss: 0.0453 - val_acc: 0.9827
Epoch 8/50
86268/86268 [==============================] - 63s 729us/step - loss: 0.0402 - acc: 0.9845 - val_loss: 0.0424 - val_acc: 0.9838
Epoch 9/50
86268/86268 [==============================] - 63s 726us/step - loss: 0.0396 - acc: 0.9845 - val_loss: 0.0442 - val_acc: 0.9842
Epoch 10/50
86268/86268 [==============================] - 63s 726us/step - loss: 0.0379 - acc: 0.9850 - val_loss: 0.0426 - val_acc: 0.9841
Epoch 11/50
86268/86268 [==============================] - 63s 730us/step - loss: 0.0367 - acc: 0.9857 - val_loss: 0.0415 - val_acc: 0.9839
Epoch 12/50
86268/86268 [==============================] - 63s 730us/step - loss: 0.0350 - acc: 0.9861 - val_loss: 0.0433 - val_acc: 0.9835
Epoch 13/50
86268/86268 

86016/86271 [============================>.] - ETA: 0s - loss: 0.0211 - acc: 0.9914Epoch 00022: val_loss did not improve

Epoch 00022: reducing learning rate to 0.0001250000059371814.
9580/9580 [==============================] - 2s 229us/step
Loss on fold 7 is 0.040522
226998/226998 [==============================] - 51s 223us/step
Training on fold 8
Train on 86275 samples, validate on 9576 samples
Epoch 1/50
86275/86275 [==============================] - 63s 732us/step - loss: 0.2523 - acc: 0.9373 - val_loss: 0.1162 - val_acc: 0.9642
Epoch 2/50
86275/86275 [==============================] - 63s 731us/step - loss: 0.0565 - acc: 0.9803 - val_loss: 0.1018 - val_acc: 0.9661
Epoch 3/50
86275/86275 [==============================] - 63s 731us/step - loss: 0.0501 - acc: 0.9817 - val_loss: 0.0748 - val_acc: 0.9739
Epoch 4/50
86275/86275 [==============================] - 63s 731us/step - loss: 0.0466 - acc: 0.9824 - val_loss: 0.0501 - val_acc: 0.9815
Epoch 5/50
86275/86275 [==================

86277/86277 [==============================] - 64s 737us/step - loss: 0.0401 - acc: 0.9844 - val_loss: 0.0452 - val_acc: 0.9831
Epoch 9/50
86277/86277 [==============================] - 62s 722us/step - loss: 0.0390 - acc: 0.9849 - val_loss: 0.0612 - val_acc: 0.9826
Epoch 10/50
86277/86277 [==============================] - 63s 733us/step - loss: 0.0376 - acc: 0.9852 - val_loss: 0.0449 - val_acc: 0.9832
Epoch 11/50
86277/86277 [==============================] - 63s 729us/step - loss: 0.0363 - acc: 0.9858 - val_loss: 0.0457 - val_acc: 0.9838
Epoch 12/50
86277/86277 [==============================] - 63s 727us/step - loss: 0.0351 - acc: 0.9860 - val_loss: 0.0458 - val_acc: 0.9831
Epoch 13/50
86277/86277 [==============================] - 63s 731us/step - loss: 0.0338 - acc: 0.9865 - val_loss: 0.0442 - val_acc: 0.9827
Epoch 14/50
86277/86277 [==============================] - 63s 726us/step - loss: 0.0324 - acc: 0.9871 - val_loss: 0.0561 - val_acc: 0.9814
Epoch 15/50
86277/86277 [========